# 1.1 Data Examination and Feature Engineering

In [1]:
import pandas as pd
from datetime import datetime
from uszipcode import SearchEngine

/opt/homebrew/Caskroom/miniforge/base/envs/deeprec/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


We begin by reading all three files:

In [2]:
ratings = pd.read_csv("../data/ratings.csv", header=0)
users = pd.read_csv("../data/users.csv", header=0)
movies = pd.read_csv("../data/movies.csv", header=0, encoding="latin-1")

### Movies table
Let's take alook at the movies table first:

In [3]:
movies.head()

,movie,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
movies.isna().any()

movie     False
title     False
genres    False
dtype: bool

We can clearly see that both the movie's title and the year it was released are stored under `title`. We'll split these two into different features. We also see that the `genres` feature is a list of all matching genres, separated by `|`. We can split it too.

In [5]:
movies[['title', 'movie_year', 'genres']] = movies.apply(lambda row: pd.Series({
    'title': row['title'][:-7],
    'movie_year': int(row['title'][-5:-1]),
    'genres': str(row['genres']).split('|') if not pd.isnull(row['genres']) else list()
}), axis=1)
movies.head()

,movie,title,genres,movie_year
0,1,Toy Story,"[Animation, Children's, Comedy]",1995
1,2,Jumanji,"[Adventure, Children's, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama]",1995
4,5,Father of the Bride Part II,[Comedy],1995


In [13]:
movies['movie_year'].unique()

array([1995, 1994, 1996, 1976, 1993, 1992, 1988, 1967, 1964, 1977, 1965,
       1982, 1962, 1990, 1991, 1989, 1937, 1940, 1969, 1981, 1973, 1970,
       1960, 1955, 1956, 1959, 1968, 1980, 1975, 1986, 1948, 1943, 1963,
       1950, 1946, 1987, 1997, 1974, 1958, 1949, 1972, 1998, 1933, 1952,
       1951, 1957, 1961, 1954, 1934, 1944, 1942, 1941, 1953, 1939, 1947,
       1945, 1938, 1935, 1936, 1926, 1932, 1930, 1971, 1979, 1966, 1978,
       1985, 1983, 1984, 1931, 1922, 1927, 1929, 1928, 1925, 1923, 1999,
       1919, 2000, 1920, 1921])

### Ratings table
Let's take a look at the ratings data:

In [6]:
ratings.head()

,user,movie,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [7]:
ratings.isna().any()

user         False
movie        False
rating       False
timestamp    False
dtype: bool

The time the user rated the movie is expressed as an [_epoch_](https://www.epochconverter.com/). We'll translate it to the three features which it is reasonable to believe will hav ean impact on a user's preference:

* hour of the day
* day of the week
* month

We'll extract those from the `timestamp` feature.

In [8]:
def features_from_epoch(row):
    dt = datetime.fromtimestamp(row['timestamp'])
    return pd.Series({
        'hour': int(dt.strftime("%H")),
        'day': int(dt.strftime("%w")),
        'month': int(dt.strftime("%m"))
    })

ratings[['hour','day','month']] = ratings.apply(features_from_epoch, axis=1)
ratings.head()

,user,movie,rating,timestamp,hour,day,month
0,1,1193,5,978300760,0,1,1
1,1,661,3,978302109,0,1,1
2,1,914,3,978301968,0,1,1
3,1,3408,4,978300275,0,1,1
4,1,2355,5,978824291,1,0,1


### Users table
Finally, the users table:

In [9]:
users.head()

,user,gender,age,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [10]:
users.isna().any()

user          False
gender        False
age           False
occupation    False
zip           False
dtype: bool

First, let's consider the `gender` feature. There are 2 possible values: `M`, `F`, stores as a string. For simplicity, we'll transform these to numbers - `1` for `F` and `0` for `M`.

Next, let's take a look at the `zip` feature. Note that all users who provided zip codes are from the US, and therefore the first thing we'll have to do is clean the zip codes and keep only the first 5 digits (Non-US students can read about the US ZIP Code system [here](https://en.wikipedia.org/wiki/ZIP_Code)). We can therefore extract the city and state from the zip code. For this, we'll use the `uszipcode` library and it's `SearchEngine` (running this for the first time requires network connection).

In [11]:
zip_code_search = SearchEngine()

In [12]:
def user_features(row):
    zip_code = int(str(row['zip'])[:5]) if not pd.isnull(row['zip']) else 0
    z = zip_code_search.by_zipcode(zip_code).to_dict()
    return pd.Series({
        'gender': int(row['gender'] == 'F'),
        'city': z.get('major_city', ''),
        'state': z.get('state', ''),
        'zip': zip_code
    })

users[['gender', 'city', 'state', 'zip']] = users.apply(user_features, axis=1)
users.head()

,user,gender,age,occupation,zip,city,state
0,1,1.0,1,10,48067.0,Royal Oak,MI
1,2,0.0,56,16,70072.0,Marrero,LA
2,3,0.0,25,15,55117.0,Saint Paul,MN
3,4,0.0,45,7,2460.0,Newtonville,MA
4,5,0.0,25,20,55455.0,Minneapolis,MN


### Joined table
Next, we'll join all three tables to a single dataset, and handle missing values. We'll also keep the `timestamp` column to use it to split the dataset to train and test data

In [13]:
all_data = ratings.join(users.set_index('user'), on='user', how='inner', rsuffix='2').join(movies.set_index('movie'), on='movie', how='inner', rsuffix='2')
print(f'Samples count: {len(all_data.index)}')
all_data.head()

Samples count: 1000209


,user,movie,rating,timestamp,hour,day,month,gender,age,occupation,zip,city,state,title,genres,movie_year
0,1,1193,5,978300760,0,1,1,1.0,1,10,48067.0,Royal Oak,MI,One Flew Over the Cuckoo's Nest,[Drama],1975
120,2,1193,5,978298413,23,0,12,0.0,56,16,70072.0,Marrero,LA,One Flew Over the Cuckoo's Nest,[Drama],1975
1339,12,1193,4,978220179,1,0,12,0.0,25,12,32793.0,Winter Park,FL,One Flew Over the Cuckoo's Nest,[Drama],1975
1518,15,1193,4,978199279,20,6,12,0.0,25,7,22903.0,Charlottesville,VA,One Flew Over the Cuckoo's Nest,[Drama],1975
1747,17,1193,5,978158471,8,6,12,0.0,50,1,95350.0,Modesto,CA,One Flew Over the Cuckoo's Nest,[Drama],1975


In [14]:
all_data.isna().any()

user          False
movie         False
rating        False
timestamp     False
hour          False
day           False
month         False
gender        False
age           False
occupation    False
zip           False
city           True
state          True
title         False
genres        False
movie_year    False
dtype: bool

We see we have no missing values except for some `city` and `state` calues, due to zip codes which `SearchEngine` couldn't locate.

In [15]:
string_na = 'XX'
all_data[['city', 'state']] = all_data[['city', 'state']].fillna(string_na)
all_data.isna().any()

user          False
movie         False
rating        False
timestamp     False
hour          False
day           False
month         False
gender        False
age           False
occupation    False
zip           False
city          False
state         False
title         False
genres        False
movie_year    False
dtype: bool

In [16]:
all_data.head()

,user,movie,rating,timestamp,hour,day,month,gender,age,occupation,zip,city,state,title,genres,movie_year
0,1,1193,5,978300760,0,1,1,1.0,1,10,48067.0,Royal Oak,MI,One Flew Over the Cuckoo's Nest,[Drama],1975
120,2,1193,5,978298413,23,0,12,0.0,56,16,70072.0,Marrero,LA,One Flew Over the Cuckoo's Nest,[Drama],1975
1339,12,1193,4,978220179,1,0,12,0.0,25,12,32793.0,Winter Park,FL,One Flew Over the Cuckoo's Nest,[Drama],1975
1518,15,1193,4,978199279,20,6,12,0.0,25,7,22903.0,Charlottesville,VA,One Flew Over the Cuckoo's Nest,[Drama],1975
1747,17,1193,5,978158471,8,6,12,0.0,50,1,95350.0,Modesto,CA,One Flew Over the Cuckoo's Nest,[Drama],1975


Save our raw data as CSV:

In [17]:
all_data.to_csv('../local_data/all_data.csv', index=False)